In [44]:
from keybert import KeyBERT
from transformers import RobertaTokenizer, RobertaModel
import transformers
import nltk
from keyphrase_vectorizers import KeyphraseCountVectorizer
from pdfminer.high_level import extract_text
import re

In [45]:
def extract_pdf(filename):
    text = extract_text(filename)
    return text

In [46]:
def filter_gutenberg_parts(paragraphs):
    # Define regular expressions to match Gutenberg-related parts
    gutenberg_regex = r'[Gg][Uu][Tt][Ee][Nn][Bb][Ee][Rr][Gg]'

    # Remove Gutenberg-related parts from each paragraph
    filtered_paragraphs = []
    for paragraph in paragraphs:
        if not re.search(gutenberg_regex, paragraph):
            filtered_paragraphs.append(paragraph)

    return filtered_paragraphs

In [47]:
def foreword_split(text):
    foreword_regex = r'[Ff][Oo][Rr][Ee][Ww][Oo][Rr][Dd]'
    if re.search(foreword_regex, text) != None:
        return text.split(re.search(foreword_regex, text).group(0))[1]
    return text

In [48]:
def splitParagraph(paragraphs):
    i = 0
    while i < len(paragraphs):
        current_paragraph = paragraphs[i]
        current_paragraph_word_count = len(current_paragraph.split())

        if current_paragraph_word_count <= 50:
            # Check the previous paragraph
            if i > 0 and i < len(paragraphs) - 1:
                if len(paragraphs[i-1].split()) <= len(paragraphs[i+1].split()):
                    merged_paragraph = paragraphs[i-1] + "\n" + current_paragraph
                    paragraphs[i-1] = merged_paragraph
                    del paragraphs[i]
                    continue
                else:
                    merged_paragraph = current_paragraph + "\n" + paragraphs[i+1]
                    paragraphs[i+1] = merged_paragraph
                    del paragraphs[i]
                    continue
            elif i == 0:
                merged_paragraph = current_paragraph + "\n" + paragraphs[1]
                paragraphs[1] = merged_paragraph
                del paragraphs[0]
                continue
            elif i == len(paragraphs) - 1:
                merged_paragraph = paragraphs[i-1] + "\n" + current_paragraph
                paragraphs[i-1] = merged_paragraph
                del paragraphs[i]
                continue


        i += 1

In [49]:
def findRelevantParagraphs(paragraphs, keywords,text,model,stop_words,n = 10,relevant_paragraphs = []):
    for paragraph in paragraphs:
        count_keywords = 0
        for keyword in keywords:
            if keyword[0] in paragraph:
                count_keywords += 1
        if count_keywords >= 2 and paragraph not in relevant_paragraphs:
            relevant_paragraphs.append(paragraph)
    
    if(len(relevant_paragraphs) < 10):        
        keywords = model.extract_keywords(text, keyphrase_ngram_range=(1,1), diversity=0.2,top_n=n+5,stop_words=stop_words,use_maxsum=True,use_mmr=True)
        findRelevantParagraphs(paragraphs,keywords,text,model,stop_words,n+5,relevant_paragraphs)
    return relevant_paragraphs
        

In [50]:
# Getting rid of the Gutenberg parts
text = extract_pdf('pdfs\\The_WonderfulWizardofOz.pdf')
paragraphs = text.split("\n\n")
splitParagraph(paragraphs)
paragraphs = filter_gutenberg_parts(paragraphs)
text = '\n\n'.join(paragraphs)
text = foreword_split(text)
paragraphs = text.split("\n\n")

In [51]:
stop_words = [
    "a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", 
    "are", "aren", "as", "at", "be", "because", "been", "before", "being", "below", "between", 
    "both", "but", "by", "can", "couldn", "d", "did", "didn", "do", "does", "doesn", "doing", "don", 
    "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "has", "hasn", "have", 
    "haven", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", 
    "i", "if", "in", "into", "is", "isn", "it", "itself", "just", "ll", "m", "ma", "me", "mightn", 
    "more", "most", "mustn", "my", "myself", "needn", "no", "nor", "not", "now", "o", "of", "off", 
    "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", 
    "s", "same", "shan", "she", "should", "shouldn", "so", "some", "such", "t", "than", "that", 
    "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", 
    "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "we", 
    "were", "weren", "what", "when", "where", "which", "while", "who", "whom", "why", "will", 
    "with", "wouldn", "y", "you", "your", "yours", "yourself", "yourselves","gutenberg","ebook",
    "author","ebooks","illustrated","illustrated","manuscript","literature","book","proofreading",
    "books","illustrations","project","online","edition","title","release","rights","reserved",
    "editions","edition","chapter","chapters","contents","contents","table","contents","table",
    "proofreaders","_italic","italic","punctuation","punctuation","transcriber","pretext"
]

In [52]:
kw_model = KeyBERT()
keywordsKeybert = kw_model.extract_keywords(text, keyphrase_ngram_range=(1,1), diversity=0.2,top_n=5,stop_words=stop_words,use_maxsum=True,use_mmr=True)
print(keywordsKeybert)

[('dorothy', 0.4767), ('oz', 0.4539), ('scarecrows', 0.4238), ('grimm', 0.4144), ('folklore', 0.4092), ('tales', 0.3862), ('wizard', 0.365), ('witches', 0.3642), ('glinda', 0.3557), ('lion', 0.3339)]


In [53]:
relevant_paragraphs = findRelevantParagraphs(paragraphs,keywordsKeybert,text,kw_model,stop_words)

In [54]:
relevant_paragraphs = sorted(relevant_paragraphs, key=lambda p: sum([1 for keyword in keywordsKeybert if keyword[0] in p]), reverse=True)

In [56]:
model_name = 'roberta-base'
keybert_model = KeyBERT(model=model_name)

No sentence-transformers model found with name C:\Users\xraef/.cache\torch\sentence_transformers\roberta-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\xraef/.cache\torch\sentence_transformers\roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
keyword_dict = {}

for paragraph in relevant_paragraphs:
    try:
        keywordsRoberta = keybert_model.extract_keywords(paragraph, keyphrase_ngram_range=(1, 1), stop_words=stop_words, use_maxsum=True, top_n=5, use_mmr=True, diversity=0.2,vectorizer=KeyphraseCountVectorizer())
    except:
        keywordsRoberta = kw_model.extract_keywords(paragraph, keyphrase_ngram_range=(1, 1), stop_words=stop_words, use_maxsum=True, top_n=5, use_mmr=True, diversity=0.2,vectorizer=KeyphraseCountVectorizer())
    
    keyword_dict[tuple(keywordsRoberta)] = paragraph

# Print the keywords and corresponding paragraphs
for keywords, paragraph in keyword_dict.items():
    print("Keywords:", keywords)
    print("Paragraph:", paragraph)
    print()


Keywords: (('civilized countries', 0.9152), ('sorceresses', 0.9), ('witches', 0.8938), ('wizards', 0.8897), ('magicians', 0.8827))
Paragraph: “Oh, yes,” replied Dorothy.
“Then that accounts for it. In the civilized countries I believe there
are no witches left, nor wizards, nor sorceresses, nor magicians. But,
you see, the Land of Oz has never been civilized, for we are cut off
from all the rest of the world. Therefore we still have witches and
wizards amongst us.”

Keywords: (('old time fairy tale', 0.9294), ('other human creations', 0.9201), ('fairy tales', 0.9168), ('instinctive love', 0.9165), ('disagreeable incident', 0.9095))
Paragraph: Introduction
Folklore, legends, myths and fairy tales have followed childhood
through the ages, for every healthy youngster has a wholesome and
instinctive love for stories fantastic, marvelous and manifestly
unreal. The winged fairies of Grimm and Andersen have brought more
happiness to childish hearts than all other human creations.
Yet the old 

In [58]:
# Tokenize the paragraphs into sentences
nltk.download('punkt')

sentences = []
for para in relevant_paragraphs:
    para_sentences = nltk.sent_tokenize(para)
    sentences.append(para_sentences)

# Tokenize the sentences into words
words = []
for sentence in sentences:
    sentence_words = nltk.word_tokenize(" ".join(sentence))
    words.append(sentence_words)
    print(sentence_words)

# Output the words (including punctuation) in an array
output_array = words

['“', 'Oh', ',', 'yes', ',', '”', 'replied', 'Dorothy', '.', '“', 'Then', 'that', 'accounts', 'for', 'it', '.', 'In', 'the', 'civilized', 'countries', 'I', 'believe', 'there', 'are', 'no', 'witches', 'left', ',', 'nor', 'wizards', ',', 'nor', 'sorceresses', ',', 'nor', 'magicians', '.', 'But', ',', 'you', 'see', ',', 'the', 'Land', 'of', 'Oz', 'has', 'never', 'been', 'civilized', ',', 'for', 'we', 'are', 'cut', 'off', 'from', 'all', 'the', 'rest', 'of', 'the', 'world', '.', 'Therefore', 'we', 'still', 'have', 'witches', 'and', 'wizards', 'amongst', 'us', '.', '”']
['Introduction', 'Folklore', ',', 'legends', ',', 'myths', 'and', 'fairy', 'tales', 'have', 'followed', 'childhood', 'through', 'the', 'ages', ',', 'for', 'every', 'healthy', 'youngster', 'has', 'a', 'wholesome', 'and', 'instinctive', 'love', 'for', 'stories', 'fantastic', ',', 'marvelous', 'and', 'manifestly', 'unreal', '.', 'The', 'winged', 'fairies', 'of', 'Grimm', 'and', 'Andersen', 'have', 'brought', 'more', 'happiness',

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xraef\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
